In [2]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from shapely.geometry import Point
from tqdm import tqdm

### Wczytanie danych

In [2]:
df = pd.read_csv('manicure_hybrydowy.csv', encoding='utf-8', sep=';')

In [3]:
df.head()

,Unnamed: 0,Description,Duration,Location,Price
0,0,Manicure hybrydowy,2g,"Generała Władysława Sikorskiego 14, 09-410, Płock",120.00
1,1,Manicure hybrydowy+ baby boomer,3g,"Generała Władysława Sikorskiego 14, 09-410, Płock",170.00
2,2,Manicure hybrydowy+ french,2g,"Generała Władysława Sikorskiego 14, 09-410, Płock",150.00
3,3,Manicure hybrydowy+ baza proteinowa,1g 30min,"ul. Fr. Nullo 42 lok U11, 31-543, Kraków, Śród...",130.00
4,4,Manicure hybrydowy,1g 30min,"ul. Fr. Nullo 42 lok U11, 31-543, Kraków, Śród...",110.00


In [4]:
df = df.drop(columns=['Unnamed: 0'], axis=1)

In [5]:
df.head()

,Description,Duration,Location,Price
0,Manicure hybrydowy,2g,"Generała Władysława Sikorskiego 14, 09-410, Płock",120.00
1,Manicure hybrydowy+ baby boomer,3g,"Generała Władysława Sikorskiego 14, 09-410, Płock",170.00
2,Manicure hybrydowy+ french,2g,"Generała Władysława Sikorskiego 14, 09-410, Płock",150.00
3,Manicure hybrydowy+ baza proteinowa,1g 30min,"ul. Fr. Nullo 42 lok U11, 31-543, Kraków, Śród...",130.00
4,Manicure hybrydowy,1g 30min,"ul. Fr. Nullo 42 lok U11, 31-543, Kraków, Śród...",110.00


### Ograniczenie zbioru danych do danych zawierających 'Laminację rzęs' oraz posiadających konkretną cenę

In [12]:
data = df[df['Description'].str.contains('Manicure hybrydowy')]

In [13]:
data = data[(data['Price'] != '-') & (data['Price'] != 'Zmienna')& (data['Price'] != 'Darmowa')]
data.reset_index(inplace=True)
data = data.drop(columns=['index'], axis=1)
data['Price'] = data['Price'].astype(float)

In [17]:
data.head()

,Description,Duration,Location,Price
0,Manicure hybrydowy,2g,"Generała Władysława Sikorskiego 14, 09-410, Płock",120.0
1,Manicure hybrydowy+ baby boomer,3g,"Generała Władysława Sikorskiego 14, 09-410, Płock",170.0
2,Manicure hybrydowy+ french,2g,"Generała Władysława Sikorskiego 14, 09-410, Płock",150.0
3,Manicure hybrydowy+ baza proteinowa,1g 30min,"ul. Fr. Nullo 42 lok U11, 31-543, Kraków, Śród...",130.0
4,Manicure hybrydowy,1g 30min,"ul. Fr. Nullo 42 lok U11, 31-543, Kraków, Śród...",110.0


In [19]:
data.shape

(14004, 4)

### Zmiana adresu na dane geolokalizacyjne

In [20]:
geolocator = Nominatim(user_agent="PDP/7")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=5)  # Rate limiter to avoid overloading the service

In [21]:
def geocode_address(address):
    try:
        location = geolocator.geocode(address)
        if location:
            return Point(location.longitude, location.latitude)
    except:
        return 'brak'

In [23]:
tqdm.pandas()
data['geometry'] = data['Location'].progress_apply(geocode_address)

  0%|          | 0/14004 [00:00<?, ?it/s]

100%|██████████| 14004/14004 [2:04:15<00:00,  1.88it/s]  


### Sprawdzenie czy wszystkie adresy zostały porzekształcone w dane geolokalizacyjne

In [42]:
data[data['geometry']=='brak']

,Description,Duration,Location,Price,geometry
9866,Manicure hybrydowy bez koloru,1g,"plac Kazimierza Pułaskiego 7, C.H Manhattan, k...",100.0,brak
10465,Manicure hybrydowy,2g,"rynek zygmunta augusta 12/1, 16-300, Augustów",120.0,brak


In [43]:
data[data['geometry'].isna()].shape

(7421, 5)

In [64]:
data = data[(data['geometry'] != 'brak') & (data['geometry'].notna())]
data.reset_index(inplace=True)
data = data.drop(columns=['index'], axis=1)

In [65]:
data

,Description,Duration,Location,Price,geometry
0,Manicure hybrydowy,2g,"Generała Władysława Sikorskiego 14, 09-410, Płock",120.0,POINT (19.7551731 52.5378894)
1,Manicure hybrydowy+ baby boomer,3g,"Generała Władysława Sikorskiego 14, 09-410, Płock",170.0,POINT (19.7551731 52.5378894)
2,Manicure hybrydowy+ french,2g,"Generała Władysława Sikorskiego 14, 09-410, Płock",150.0,POINT (19.7551731 52.5378894)
3,Manicure hybrydowy ( bez zdjęcia poprzedniego ...,1g 15min,"Świerkowa 36, 18-230, Ciechanowiec",100.0,POINT (22.4891183 52.6904013)
4,Manicure hybrydowy +henna brwi+ regulacja+ dep...,1g 30min,"Świerkowa 36, 18-230, Ciechanowiec",155.0,POINT (22.4891183 52.6904013)
...,...,...,...,...,...
6576,Manicure hybrydowy,1g 30min,"Handlowa 13/4, 84-300, Lębork",80.0,POINT (17.7509876 54.5398359)
6577,Manicure hybrydowy,1g 30min,"Targowa 1, 41-710, Ruda Śląska",120.0,POINT (18.86451695265776 50.2733113)
6578,Manicure hybrydowy,1g,"Juliusza Słowackiego 2, 32-050, Skawina",115.0,POINT (19.827164 49.9748503)
6579,Manicure hybrydowy,1g,"Insurekcji Kościuszkowskiej, 47, 43-600, Jaworzno",100.0,POINT (19.289151387569486 50.195196800000005)


In [59]:
data.shape

(6581, 6)

### Usunięcie podwójnych rekordów

In [10]:
data.shape

(6581, 5)

In [11]:
data = data.drop_duplicates()

In [12]:
data.shape

(4536, 5)

### Zapisanie danych do pliku pickle

In [13]:
data.to_pickle('data_hybrid.pkl')